<a href="https://colab.research.google.com/github/brkuhgk/yolo/blob/master/YoloVedioCap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
from google.colab.patches import cv2_imshow

In [141]:
import cv2
import numpy as np
import os
import math

In [142]:
#drive/MyDrive/new/yolo/

# Load Yolo 
#drive/MyDrive/new/yolo/
net = cv2.dnn.readNet("drive/MyDrive/new/yolo/yolov3.weights", "drive/MyDrive/new/yolo/yolov3.cfg")
classes = []
with open("drive/MyDrive/new/yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [143]:
# myset=set()
# for id in class_ids:
#   myset.add(id)
# myset

#firstFrame
firstFrame =None
first=1
first2 =1
warn =0; #variable to indicate warning

In [144]:
#vediocapture
vediocap = cv2.VideoCapture('drive/MyDrive/new/yolo/chair2.mp4')
vediocap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
vediocap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
vediocap.set(cv2.CAP_PROP_FPS, 30) 

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
outer = cv2.VideoWriter('char2.avi',fourcc, 30.0, (640,480))


In [145]:
terminate =0;
fmid_x=[]
fmid_y=[]
mid_x=[]
mid_y=[]


In [146]:
def distance(x1,x2,y1,y2):
  distc = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
  return distc


In [147]:

while vediocap.isOpened():
    # Capture frame-by-frame
  ret, img = vediocap.read()
  if not ret:
    break
  # Loading image
  # img = cv2.imread("drive/MyDrive/new/yolo/")
  # img = cv2.resize(img, None, fx=0.4, fy=0.4)
  height,width,channels = img.shape
  
  #Detecting objects
  blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
  net.setInput(blob)

  outs = net.forward(output_layers)
  # Showing informations on the screen
  class_ids = []
  confidences = []
  boxes = []
  myset =set()

  for out in outs:
      for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            #here we add set
            myset.add(class_id)

#if object is Lost

  #if live we update for specific time
  if first:
    print("first frame")
    fixedset=set()
    first =0
    firstframe =img
    fixedset =[c for c in myset]
    no_of_fixedset =len(fixedset)
  
  # print("fixedSet ",len(fixedset))
  # print("myset ",len(myset))
  
  no_of_myset = len(myset)

  if(no_of_fixedset != no_of_myset):
    warn=warn+1;
  entered =0
  lost=0
  if(warn >5):
    warn=0
    #update the values
    
    print("=======================================================")
    if(no_of_fixedset < no_of_myset): #if both no. equal it will not come here
        print("New object entered ")
        entered=1
    else:
        print("object Lost")
        lost=1
    no_of_fixedset = no_of_myset




  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  kil =len(boxes)
  for j in range(kil):
    fmid_x.append(0)
    fmid_y.append(0)
    mid_x.append(0)
    mid_y.append(0)

  font = cv2.FONT_HERSHEY_PLAIN
  for i in range(len(boxes)):
      # print(i)
      k=0
      if i in indexes:
          x, y, w, h = boxes[i]
          label = str(classes[class_ids[i]])
          if(entered):
            label =label+" Entered "
            entered =0
          if(lost):
            label =label+" MISSING ! ALERT "
            lost =0
          color = colors[i]
          cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
          cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
          #if object moves
          if kil:
            fmid_x[k] =int((2*x+w)/2)
            fmid_y[k] =int((2*y+h)/2)
            kil =kil-1

          mid_x[k] =(2*x+w)/2
          mid_y[k] =(2*y+h)/2
          #find distance b/w two objects
          dist = distance(fmid_x[k],mid_x[k],fmid_y[k],mid_y[k])
          #using this we can even predict direct of object moving 

          print("distance : " + label,dist)
          k = k+1

            


  
          

  # cv2_imshow(img) #uncoment this to see the image frames
  outer.write(img)
  terminate=terminate+1

vediocap.release()
outer.release()
cv2.waitKey(0)
# cv2.destroyAllWindows()


first frame
distance : chair 0.7071067811865476
distance : book 0.7071067811865476
distance : chair 0.7071067811865476
distance : book 0.7071067811865476
distance : chair 0.5
distance : book 0.5
distance : chair 0.5
distance : book 0.5
distance : chair 0.7071067811865476
distance : book 0.5
distance : chair 0.7071067811865476
distance : book 0.0
distance : chair 0.7071067811865476
distance : book 0.5
distance : chair 0.0
distance : book 0.5
distance : chair 0.7071067811865476
distance : book 0.5
distance : chair 0.5
distance : book 0.5
distance : chair 0.0
distance : book 0.7071067811865476
distance : chair 0.7071067811865476
distance : book 0.5
distance : person 0.7071067811865476
distance : chair 0.7071067811865476
distance : book 0.0
distance : person 0.0
distance : chair 0.5
distance : book 0.0
distance : person 0.5
distance : chair 0.5
distance : book 0.0
distance : person 0.0
distance : chair 0.5
distance : person 0.0
distance : book 0.0
New object entered 
distance : chair Enter

-1